# Lab 4 - Regression Diagnostics

In this lab, we'll be analyzing the following:
1. Multicolinearity & Variance Inflation Factors (VIF)
1. Heteroskedasticity & Breusch Pagan test

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor ### VIF package
from stargazer.stargazer import Stargazer

### Load & Clean the Data

In [ ]:
raw = pd.read_csv ('data.csv')
raw.head()

Create a copy with only the variables of interest

In [ ]:
var_list = ['price_000','pop_dens','ses','house','area_m2','num_bath','pcn_green','homicides', 'year']
data = raw[var_list].copy()

data.head()

#### Recoding Variables

Let's redefine the socieconomic status - this is the same process we did in lab 3!

In [ ]:
#Create SES Dummy Variable
data['high_ses'] = np.where(data['ses']>=5, 1, 0)

Now, let's create dummy variables for the year the property was sold. There are 6 unique years, and we can create a new dummy column for each year. This could be a long process, but python can make this easier for us!

In [ ]:
#Pandas '.get_dummies()' Function
dummies = pd.get_dummies(data['year'], prefix = 'yr') 
#If we don't specify drop_first=True, python will give us a dummy column for every year
#Remember we'll always need to exclude one dummy in our regression model - it helps to think
#through which value you want to be your base comparison!

dummies.head()

In [ ]:
#Append the dummies to our larger dataframe
data = pd.concat([data, dummies], axis = 1)
data.head()

### Estimate a Multivariate Linear Regression Model

We'll be running a similiar multivariate regression model to the one we used for Lab 3

In [ ]:
#Define Independent Variables of Interest
ind_var = ['high_ses', 'house', 'area_m2', 'num_bath', 'pcn_green', 'homicides','yr_2002','yr_2003','yr_2004','yr_2005','yr_2006'] 
#Note that the year variable is categorical. We need to exclude one to prevent collinearity issues with our model
#We will exclude year 2001 - we choose to have the earlier year be our base year

x = data[ind_var].assign(Intercept = 1) #Independent Variables
y = data['price_000'] #Dependent Variable

model = sm.OLS(y, x).fit()
### Let's save the results as "model" - this will be useful for other functions below.

model.summary2()

In [ ]:
model

In [ ]:
Stargazer([model])

### 1. Examine Multicolinearity

First, let's examine the correlation matrix. If we take price to be our dependent variable - which of these variables will have a multicollinearity effect in our regression? We are looking at all the independent variables we used in our regression, defined as `ind_var`

In [ ]:
data[ind_var].corr() #Note that 'price_000' was excluded, as it's the dependent variable

Now, let's use the Variance Inflation Factor (VIF)

In [ ]:
#Remembering what our x independent variables look like to python
print(type(x))
x.head() #It's a sub-dataframe!

In [ ]:
# The VIF function does not understand what a dataframe is
print(type(x.values))
x.values

In [ ]:
vif = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])] 
#Use the x independent variables that include an intercept!
vif

Let's add some labels to the data.

In [ ]:
pd.Series(vif, index=x.columns)

The variance_inflation_factor function can be confusing, we know. We wish it were simpler, we really do. The good thing is, you'll have it coded forever! The bad thing is, well... See above. If any of this is still unclear, or your eyes are glazing over the above, please come to office hours. We'll do our best to walk you through it, as many times as you might need it.

### 2. Examine Homoscedasticity

Before jumping right into this, let's looks at predicted values. To do this, we'll need the results of the regression model that we ran above.

Remember we saved the results of the linear regression under the "model" variable above:  
> model = sm.OLS(y, x).fit()

In [ ]:
model.params # This will give us a summary of the regression coefficients.

We can rewrite this into an equation form, as shown below:

$y = 27215.4 * high\_ses\; \;- 25760.7 * house\; \;+566.9 * area\_m2\; \;+10559.7*num\_bath\; \;+161.1*pcn\_green\;\;-24.0*homicides$ <br> $-265.75*yr\_2002\;\;+8079.5*yr\_2003\;\;+8448.1*yr\_2004\;\;18176.6*yr\_2005\;\;+23245.9*yr\_2006\;\;-4589.5$

Now, let's try our first prediction! To do this, we are going to use the first line of observed data from the dataframe, as shown below:

In [ ]:
x.head(1)

$y = 27 215.4 * 0\; \;- 25760.7 * 0\; \;+566.9 * 70\; \;+10559.7*2\; \;+161.1*1.74\;\;-24.0*39.92$ <br> $-265.75*0\;\;+8079.5*0\;\;+8448.1*0\;\;18176.6*0\;\;+23245.9*0\;\;-4589.5$

In [ ]:
model.predict(x.iloc[0].values)

How does this compare to the observed price value for that observation?

In [ ]:
data.head(1)

What do you think the residual of this prediction is?

In [ ]:
model.resid[0]

Now, we can tell python to show us the predictions and residuals for every single observation in our dataset.

In [ ]:
model.predict()

In [ ]:
model.resid

In [ ]:
def homoscedasticity_plots(observed, prediction, residuals):

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    ### Figure 1
    x1 = prediction
    y1 = residuals
    ### Figure 2
    x2 = observed
    y2 = residuals

    ### Creates title for overall figure
    plt.suptitle('Residuals vs. observed and fitted data')

    ### Creates first plot
    ax1.scatter(x1, y1)
    ax1.set_xlabel('Predicted Values')
    ax1.set_ylabel('Residuals')
    ### Formats axis number to include thousands separator
    ax1.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x1, loc: "{:,}".format(int(x1))))
    ax1.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda y1, loc: "{:,}".format(int(y1))))

    ### Creates second plot
    ax2.scatter(x2, y2) ### We can change the color and marker type
    ax2.set_xlabel('Observed Values')

    ### Formats axis number to include thousands separator
    ax2.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x1, loc: "{:,}".format(int(x1))))
    ax2.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda y1, loc: "{:,}".format(int(y1))))

    plt.show()

In [ ]:
observed = data['price_000'] 
### These are the observed values of prices

prediction = model.predict()
### These are the price predictions using the same x values as the data input

residuals = model.resid 
### These are the residuals of the prediction model

In [ ]:
homoscedasticity_plots(prediction, residuals, observed)